In [1]:
library(tidyverse)
library(lubridate)

library(ggplot2)
library(reshape2)
library(broom)
library(tidyr)
library(dplyr)

library(car)
library(GGally)
library(caTools)
library(knitr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some


Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



*This dataset lists over 215000 top projects by star with over 167 stars. The dataset was collected using a GitHub search API. This API will get the first 1000 for a query. By looping through the low to high pairs the API returns less than a thousand repositories when query = Star:{Low}...{High}. The repository includes the name, description, URL, date of creation, date of last update, homepage, size, and stars, among other attributes for a total of 24 attributes.*

- Name:               chr 
- Description:        chr
- URL:                chr 
- Created At:         dttm 
- Updated At:         dttm
- Homepage:           chr
- Size:               dbl
- Stars:              dbl
- Forks:              dbl 
- Issues:             dbl
- Watchers:           dbl
- Language:           chr 
- License:            chr 
- Topics:             chr 
- Has Issues:        lgl
- Has Projects:      lgl 
- Has Downloads:     lgl 
- Has Wiki:          lgl 
- Has Pages:         lgl
- Has Discussions:   lgl 
- Is Fork:           lgl
- Is Archived:       lgl 
- Is Template:       lgl 
- Default Branch:    chr

In [2]:
df <- read_csv('repositories.csv')

df_subset <- df %>% sample_n(size = 10000)

head(df_subset, n = 3)

Rows: 215029 Columns: 24
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): Name, Description, URL, Homepage, Language, License, Topics, Defau...
dbl  (5): Size, Stars, Forks, Issues, Watchers
lgl  (9): Has Issues, Has Projects, Has Downloads, Has Wiki, Has Pages, Has ...
dttm (2): Created At, Updated At

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Name,Description,URL,Created At,Updated At,Homepage,Size,Stars,Forks,Issues,⋯,Has Issues,Has Projects,Has Downloads,Has Wiki,Has Pages,Has Discussions,Is Fork,Is Archived,Is Template,Default Branch
<chr>,<chr>,<chr>,<dttm>,<dttm>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
curso-js,Arquivos criados no meu curso de Javascript.,https://github.com/luizomf/curso-js,2019-10-24 14:41:30,2023-09-25 03:48:57,NA,13679,1370,678,52,⋯,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master
ppq,PPL Quantization Tool (PPQ) is a powerful offline neural network quantization tool.,https://github.com/openppl-public/ppq,2021-12-30 11:19:31,2023-09-25 06:29:37,NA,5806,1097,196,17,⋯,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master
cartjs,A Javascript library to power cart management for Shopify themes.,https://github.com/discolabs/cartjs,2014-06-13 09:56:25,2023-09-19 23:29:50,https://cartjs.org,3542,471,85,49,⋯,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master


* What factors affect the number of stars a GitHub repository has?
* what variables are most correlated with the success of a GitHub repository?
* Can we predict the number of stars a GitHub repository will have based on the number of other input variables (e.g. # of Issues, # of Forks, # of days since it was created, ...etc)?
* How accurate are the predictions, and what is the best-fit model based on the given input data? 

*The two cells below provide additional information about the data such as the number of observations, variable names and types, and samples of the data that is included in each column.* 

In [3]:
nrow(df_subset)

glimpse(df_subset) 

colnames(df_subset) 

[1] 10000

Rows: 10,000
Columns: 24
$ Name              <chr> "curso-js", "ppq", "cartjs", "core", "orderless", "c…
$ Description       <chr> "Arquivos criados no meu curso de Javascript.", "PPL…
$ URL               <chr> "https://github.com/luizomf/curso-js", "https://gith…
$ `Created At`      <dttm> 2019-10-24 14:41:30, 2021-12-30 11:19:31, 2014-06-1…
$ `Updated At`      <dttm> 2023-09-25 03:48:57, 2023-09-25 06:29:37, 2023-09-1…
$ Homepage          <chr> NA, NA, "https://cartjs.org", NA, NA, NA, NA, "http:…
$ Size              <dbl> 13679, 5806, 3542, 8184, 356, 28902, 802, 168, 62494…
$ Stars             <dbl> 1370, 1097, 471, 180, 584, 457, 404, 2291, 168, 281,…
$ Forks             <dbl> 678, 196, 85, 66, 25, 137, 65, 228, 77, 30, 133, 224…
$ Issues            <dbl> 52, 17, 49, 6, 7, 13, 4, 18, 0, 0, 7, 2, 14, 95, 33,…
$ Watchers          <dbl> 1370, 1097, 471, 180, 584, 457, 404, 2291, 168, 281,…
$ Language          <chr> "JavaScript", "Python", "CoffeeScript", "C++", "Emac…
$ License      

[1] "Name"            "Description"     "URL"             "Created At"     
 [5] "Updated At"      "Homepage"        "Size"            "Stars"          
 [9] "Forks"           "Issues"          "Watchers"        "Language"       
[13] "License"         "Topics"          "Has Issues"      "Has Projects"   
[17] "Has Downloads"   "Has Wiki"        "Has Pages"       "Has Discussions"
[21] "Is Fork"         "Is Archived"     "Is Template"     "Default Branch"

In [4]:
str(df)

spc_tbl_ [215,029 × 24] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ Name           : chr [1:215029] "freeCodeCamp" "free-programming-books" "awesome" "996.ICU" ...
 $ Description    : chr [1:215029] "freeCodeCamp.org's open-source codebase and curriculum. Learn to code for free." ":books: Freely available programming books" "😎 Awesome lists about all kinds of interesting topics" "Repo for counting stars and contributing. Press F to pay respect to glorious developers." ...
 $ URL            : chr [1:215029] "https://github.com/freeCodeCamp/freeCodeCamp" "https://github.com/EbookFoundation/free-programming-books" "https://github.com/sindresorhus/awesome" "https://github.com/996icu/996.ICU" ...
 $ Created At     : POSIXct[1:215029], format: "2014-12-24 17:49:19" "2013-10-11 06:50:37" ...
 $ Updated At     : POSIXct[1:215029], format: "2023-09-21 11:32:33" "2023-09-21 11:09:25" ...
 $ Homepage       : chr [1:215029] "http://contribute.freecodecamp.org/" "https://ebookfoundation.github.io/fr

*By observing the varibles and their types, Now I want to create a subset that include 
only the variables that are relevant the question I am trying to asnswer with this data.*

In [ ]:
features <- c('Created At', 'Updated At', "Size", "Stars", "Forks",
              "Issues", 'Watchers', 'Has Discussions')
df_subset <- df_subset %>% select(all_of(features))

head(df_subset, n = 3)

In [ ]:
sum(is.na(df_subset))

*The variable 'Created At' and 'Updated At' are in DateTime <dttm> format. To make them easy to work with, I will convert them to numerical values representing the number of days that have elapsed since the repository was created and updated.*

In [ ]:
head(df_subset$'Created At') 

In [ ]:
df_subset$'Created At' <- as.Date(strptime(df_subset$'Created At', format = "%Y-%m-%d %H:%M:%S"))
df_subset$'Updated At' <- as.Date(strptime(df_subset$'Updated At', format = "%Y-%m-%d %H:%M:%S"))

In [ ]:
df_subset <- df_subset %>%
  mutate(repo_age_days = as.numeric(Sys.Date() - `Created At`),
         days_since_update = as.numeric(Sys.Date() - `Updated At`))

In [ ]:
head(df_subset, n = 3)

*Now, the data needs to be cleaned before we can visualize it.*

In [ ]:
df_subset2 <- df_subset %>% select(-c('Created At', 'Updated At'))

head(df_subset2, n = 3)

In [ ]:
non_na_repo_age <- df_subset2 %>% filter(!is.na(repo_age_days))
non_na_days_since_update <- df_subset2 %>% filter(!is.na(days_since_update))

head(df_subset %>% select(repo_age_days, days_since_update, n = 3))

In [ ]:
df_cleaned <- df_subset2 %>% filter(!is.na(Stars) & !is.na(Forks) & !is.na(repo_age_days))  

head(df_cleaned, n = 3)

In [ ]:
df_summary <- summary(df_cleaned) %>%
              kable()

df_summary

In [ ]:
summary_stats <- df_cleaned %>%
  summarize(
    count = n(),
    avg_stars = mean(Stars, na.rm = TRUE),
    avg_forks = mean(Forks, na.rm = TRUE),
    avg_issues = mean(Issues, na.rm = TRUE),
    avg_repo_age = mean(repo_age_days, na.rm = TRUE),
    avg_days_since_update = mean(days_since_update, na.rm = TRUE)
  )

print(summary_stats)

In [ ]:
head(df, n = 3)
nrow(df_cleaned)
sum(is.na(df_cleaned))

In [ ]:
colSums(is.na(df_cleaned))

*Now that our data is clean, I am going to perform visualization. I will start by creating a simple linear model plot for each numerical input variable againt our desired reponse variable (Stars).*

In [ ]:
colnames(df_cleaned)

In [ ]:
continuous_vars <- c("Size", "Forks", "Issues", "Watchers", "repo_age_days", "days_since_update")

model <- lm(Stars ~ ., data = df_cleaned %>% select(Stars, all_of(continuous_vars)))

print(summary(model))

for (var in continuous_vars) {
  ggplot(df_cleaned, aes_string(x = var, y = "Stars")) +
    geom_point(alpha = 0.5) +
    geom_smooth(method = "lm", color = "red", se = FALSE) +
    labs(title = paste("Linear Model Fit for Stars vs", var)) +
    theme_minimal() -> p
  
  print(p)
}

*To investigate the presence of Multicollinearty, I am going to create a heat map to visulize
the corellation between input variables.*

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 12) 

df_cleaned_pair_plots <- 
    df_cleaned %>% 
    select(Size, Forks, Issues, Watchers, repo_age_days, days_since_update)%>% 
    ggpairs(progress = FALSE) +
    theme(text = element_text(size = 15))

df_cleaned_pair_plots

In [ ]:
corr_matrix_df_cleaned <- 
    df_cleaned %>%  
    select(Size, Forks, Issues, Watchers, repo_age_days, days_since_update) %>%
    cor() %>%
    as_tibble(rownames = 'var1') %>%
    pivot_longer(-var1, names_to = "var2", values_to = "corr")

head(corr_matrix_df_cleaned)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5) 

plot_corr_matrix_df_cleaned <- 
    corr_matrix_df_cleaned %>%
    ggplot(aes(var1, var2)) +
    geom_tile(aes(fill = corr), color = "white") +
    scale_fill_distiller("Correlation Coefficient \n",
                         palette = "YlOrRd",
                         direction = 1, 
                         limits = c(-1, 1)
    ) +
    labs(x = "", y = "") +
    theme_minimal() +
    theme(
        axis.text.x = element_text(angle = 45, vjust = 1, size = 14, hjust = 1),
        axis.text.y = element_text(vjust = 1, size = 14, hjust = 1),
        legend.title = element_text(size = 18),
        legend.text = element_text(size = 12),
        legend.key.size = unit(1.5, "cm")
    ) +
    coord_fixed() +
    geom_text(aes(var2, var1, label = round(corr, 2)), color = "black", size = 4.5)

plot_corr_matrix_df_cleaned

*We can see a strong positive correlation between 'Watchers' and 'Forks' implying multicollinearity*

In [ ]:
MLR_df_cleaned <- lm(formula = Stars ~ ., data = df_cleaned)

MLR_df_cleaned_results <- 
    tidy(MLR_df_cleaned) %>%
    mutate_if(is.numeric, round, 2)

head(MLR_df_cleaned_results)

In [ ]:
VIF_MLR_df_cleaned <- vif(MLR_df_cleaned)

round(VIF_MLR_df_cleaned, 3)

*We can see from the heat map that some of the input variables have a medium linear correlation. 
 e.g. Forks and Issues(0.667). In the next cell, we are going the find out what are their correlation coefficients.*

In [ ]:
additive_model <- lm(Stars ~ Size + Forks + Issues + Watchers + `Has Discussions`, data = df_cleaned)

additive_model_results <- tidy(additive_model, conf.int = TRUE)

additive_model
additive_model_results

In [ ]:
additive_model <- lm(Stars ~ Size * Watchers * `Has Discussions`, data = df_cleaned)

additive_model_results <- tidy(additive_model, conf.int = TRUE)

additive_model

additive_model_results